In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use default of 1 GPU

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

set working directory

In [ ]:
df_train = pd.read_csv('Data/PM25_train.csv')
df_test = pd.read_csv('Data/PM25_test.csv')
# df_sample_target = pd.read_csv('Data/sample_submission.csv')

print("Training Data Dimension: ", df_train.shape) # (7160, 184)
print("Testing Data Dimension: ", df_test.shape) # (1791, 183)

In [ ]:
# LOTs of duplicates that needs to be gotten rid of
train_len_original = len(df_train)
df_train.drop_duplicates(keep='first', inplace=True)
train_len_deduped = len(df_train)
print("Total of % duplications removed in training data!", (train_len_original - train_len_deduped))
df_train.reset_index(drop=True, inplace=True)

print("Deduped training Data Dimension: ", df_train.shape) # (7160, 184)